In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



plt.rcParams['figure.facecolor'] = 'white'
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.set(rc={'ytick.labelcolor':'black','xtick.labelcolor':'black'}) 

<h1> EDA <h1>

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    print('Unique values in column %s' %col,'\n', df[col].unique(),
        '\nNumber of unique values in column %s :' %col, df[col].nunique())

<h3> Doing one-hot encoding to be able to explore more in future data plotting <h3>

In [ ]:
dummies_sex = pd.get_dummies(df['sex'], prefix='sex')
dummies_smoker = pd.get_dummies(df['smoker'], prefix='smoker')
dummies_region = pd.get_dummies(df['region'], prefix='region')

In [ ]:
cols = ['sex', 'smoker', 'region']
dfs = [df, dummies_sex, dummies_smoker, dummies_region]
df = pd.concat(dfs, axis = 1)
df = df.drop(columns = cols)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

<h2> Visualizations <h2>

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.boxplot(x = col, data = df)
               

In [ ]:
for i, col in enumerate(df.columns):
    plt.figure(i)
    sns.histplot(x = col, data = df)

In [ ]:
for i, col in enumerate(df.columns):
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        plt.figure(i)
        sns.boxplot(x = col, data = df)

In [ ]:
sns.pairplot(df)

<h2>  Thoughts after performing EDA <h2>
<h3>. <h3>